In [1]:
from google.colab import drive
drive.mount('/content/drive')
# cd '/content/drive/Shared drives/CIS522'

Mounted at /content/drive


In [2]:
!pip install -q contractions

import pandas as pd 
import nltk
import re
import matplotlib.pyplot as plt
import contractions



nltk.download('punkt')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 4.4 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Downloading Dataset

In [ ]:
# from google.colab import files

# # TODO: UPLOAD kaggle.json file - download it frm kaggle.com if you dont have
# files.upload() #this will prompt you to upload the kaggle.json


# !ls -lha kaggle.json
# !pip install -q kaggle

In [ ]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/

# # we need to set permissions
# !chmod 600 /root/.kaggle/kaggle.json

# # check your directory before downloading the datasets
# !pwd

# # list all available datasets
# !kaggle datasets list -s stackoverflow

# !kaggle datasets download -d michaelfumery/stackoverflow-questions-filtered-2011-2021

#### Loading in Dataset - combines 2009-2020

### Preprocessing

1) Creation date to datetime
2) Separate Tags 

3) Make text lowercase 
4) Remove punctuation
5) Remove tags + extract url
6) Remove stop words
7) Encode the text

8) Remove links/hyperlinks?
9) Remove html tags?
10) Remove whitespace
11) Remove consecutive '\n's


Feature engineering: 
12) Add post length

x) Tokenize the text


TODO / QUESTIONS?
* What type of puncutation should we remove?
* How to deal with URLs in post? If a post has same URL link that's a good thing. Also if a URL is another Stackoverflow post, what to do?
* How to deal with coding syntax?

In [3]:
years = [i for i in range(2009, 2020)]


for year in years: 
  path = ("/content/drive/Shareddrives/CIS522/stackoverflow-questions-filtered-2011-2021/StackOverflow_questions_%d.csv" % year)
  if year == 2009:
    df = pd.read_csv(path)
  else: 
    df_new = pd.read_csv(path)
    df = pd.concat([df, df_new])

df.shape

(101611, 9)

In [4]:
df.head()

,Id,CreationDate,Title,Body,Tags,ViewCount,CommentCount,AnswerCount,Score
0,414336,2009-01-05 20:09:49,Why does the SqlServer optimizer get so confus...,<p>I know this has something to do with parame...,<sql><sql-server><optimization>,3368,6,9,7
1,414605,2009-01-05 21:33:21,Fluent interface for rendering HTML,<p>Rendering HTML with the HtmlTextWriter isn'...,<.net><html><render><web-controls><htmltextwri...,4557,7,4,10
2,414714,2009-01-05 22:21:04,Compiling with g++ using multiple cores,<p>Quick question: what is the compiler flag t...,<c++><compiler-construction><makefile><g++><mu...,159848,7,8,181
3,415452,2009-01-06 04:26:16,Object-orientation in C,<p>What would be a set of nifty preprocessor h...,<c><oop><object>,82624,6,23,161
4,415515,2009-01-06 04:55:55,How can I read and manipulate CSV file data in...,"<p>Pretty self-explanatory, I tried google and...",<c++><csv>,147860,7,9,51


In [5]:
num_posts = df.shape[0]

print("number of posts: %d" % num_posts)

number of posts: 101611


####1) Change to creation date to datetime

In [6]:
# 1
df['CreationDate'] = pd.to_datetime(df['CreationDate'])

In [7]:
df.head(2)

,Id,CreationDate,Title,Body,Tags,ViewCount,CommentCount,AnswerCount,Score
0,414336,2009-01-05 20:09:49,Why does the SqlServer optimizer get so confus...,<p>I know this has something to do with parame...,<sql><sql-server><optimization>,3368,6,9,7
1,414605,2009-01-05 21:33:21,Fluent interface for rendering HTML,<p>Rendering HTML with the HtmlTextWriter isn'...,<.net><html><render><web-controls><htmltextwri...,4557,7,4,10


####2) Parse tags 

In [8]:
# 2
df['Tags'] = df['Tags'].apply(lambda tags_str: [tag.split(">")[0] for tag in tags_str.split("<")[1:]])

#### 3) Change body and title to lowercase 

In [9]:
# 3
df['Title'] = df['Title'].str.lower()
df['Body'] = df['Body'].str.lower()

#### 3) Remove contractions

In [10]:
def expand_contractions(text):
    text = contractions.fix(text)
    return text


In [11]:
df['Title'] = df['Title'].apply(lambda x: expand_contractions(x)) 
df['Body'] = df['Body'].apply(lambda x: expand_contractions(x))

#### 4) Remove puncutation 

In [12]:
# 4 - punc
df['Title']

0       why does the sqlserver optimizer get so confus...
1                     fluent interface for rendering html
2                 compiling with g++ using multiple cores
3                                 object-orientation in c
4       how can i read and manipulate csv file data in...
                              ...                        
3408    how to use npm package as normal javascript in...
3409    warning:tensorflow:sample_weight modes were co...
3410           trouble with cors policy and .net core 3.1
3411    how to abort only function on error, but do no...
3412                 file upload using foreach in laravel
Name: Title, Length: 101611, dtype: object

In [13]:
df.columns

Index(['Id', 'CreationDate', 'Title', 'Body', 'Tags', 'ViewCount',
       'CommentCount', 'AnswerCount', 'Score'],
      dtype='object')

#### 5) remove html tags and extract URLs


In [14]:
url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
href_pattern = r'<a href="([^"]+)">([^<]+)<\/a>'
p_pattern = r'<\/?p[^>]*>'


# extract urls
df['urls'] = df['Body'].apply(lambda x: re.findall(url_pattern, x)) 

# remove href tags + url 
df['Clean Body'] = df['Body'].apply(lambda text: re.sub(href_pattern, '', text))

# remove paragraph tags (<p>)
df['Clean Body'] = df['Clean Body'].apply(lambda text: re.sub(p_pattern, '', text))

df.head(10)

df['urls']

0       [https://stackoverflow.com/questions/379007/kn...
1                                                      []
2                                                      []
3       [http://www.planetpdf.com/codecuts/pdfs/ooc.pd...
4                                                      []
                              ...                        
3408    [https://github.com/niklasvh/html2canvas/relea...
3409    [https://github.com/tensorflow/tensorflow/blob...
3410    [https://localhost:8082/api/auth/', http://loc...
3411                                                   []
3412                                                   []
Name: urls, Length: 101611, dtype: object

In [15]:
def show_clean_body(num_i):
  for i, x in enumerate(df['Clean Body']):
    if i == num_i:
      break
    print(i, x)

#### 6) Clean whitespace and new lines

In [16]:
df['Clean Body'] = df['Clean Body'].apply(lambda text: text.replace("\n", " "))

####7) Remove code snippets in Body

In [17]:
df['Clean Body'] = df['Clean Body'].apply(lambda text: re.sub('<.*?>', '', text))

In [18]:
show_clean_body(5)

0 i know this has something to do with parameter sniffing, but i am just perplexed at how something like the following example is even possible with a piece of technology that does so many complex things well.  many of us have run into stored procedures that intermittently run several of orders of magnitude slower than usual, and then if you copy out the sql from the procedure and use the same parameter values in a separate query window, it runs as fast as usual.  i just fixed a procedure like that by converting this:  alter procedure p_myproc (     @param1 int ) as -- do a complex query with @param1   to this:  alter procedure p_myproc (     @param1 int ) as  declare @param1copy int; set @param1copy = @param1;  -- do the query using @param1copy   it went from running in over a minute back down to under one second, like it usually runs.  this behavior seems totally random.  for 9 out of 10 @param1 inputs, the query is fast, regardless of how much data it ends up needing to crunch, or h

### Feature Engineering 


#### 12) Add post length 

In [19]:
df['body_length'] = df['Clean Body'].apply(lambda x: len(str(x)))

#### 13) Add number of code snippets 

In [20]:
# Count the number of code snippets in each post
code_regex = r'<code>[\s\S]*?<\/code>'

df['code_snippets'] = df['Body'].apply(lambda x: len(re.findall(code_regex, str(x))))

In [21]:
df['code_snippets'].head(5)

0    2
1    4
2    0
3    0
4    0
Name: code_snippets, dtype: int64

#### Tokenize text

#####Tokenize Title

In [22]:
df['clean tokenized title'] = df['Title'].apply(lambda text: nltk.word_tokenize(text))

df['clean tokenized title'].head(5)

0    [why, does, the, sqlserver, optimizer, get, so...
1            [fluent, interface, for, rendering, html]
2       [compiling, with, g++, using, multiple, cores]
3                          [object-orientation, in, c]
4    [how, can, i, read, and, manipulate, csv, file...
Name: clean tokenized title, dtype: object

In [23]:
df.shape

(101611, 14)

#####Tokenize Body (uncomment - takes 3-4 mins)



In [24]:
df['clean tokenized body'] = df['Clean Body'].apply(lambda text: nltk.word_tokenize(text))

In [25]:
df.reset_index(inplace = True)

#### Create sets

In [26]:
df['title set'] = df['clean tokenized title'].apply(lambda x: set(x))

In [27]:
df['body set'] = df['clean tokenized title'].apply(lambda x: set(x))

In [28]:
df['union body title set'] = df.apply(lambda x: x['title set'].union(x['body set']), axis=1)

In [29]:
df = df.drop(columns = ['Title', 'Body', 'urls'])

In [30]:
df.head()

,index,Id,CreationDate,Tags,ViewCount,CommentCount,AnswerCount,Score,Clean Body,body_length,code_snippets,clean tokenized title,clean tokenized body,title set,body set,union body title set
0,0,414336,2009-01-05 20:09:49,"[sql, sql-server, optimization]",3368,6,9,7,i know this has something to do with parameter...,1578,2,"[why, does, the, sqlserver, optimizer, get, so...","[i, know, this, has, something, to, do, with, ...","{sqlserver, so, does, why, confused, with, the...","{sqlserver, so, does, why, confused, with, the...","{so, why, confused, with, optimizer, get, para..."
1,1,414605,2009-01-05 21:33:21,"[.net, html, render, web-controls, htmltextwri...",4557,7,4,10,rendering html with the htmltextwriter is not ...,5117,4,"[fluent, interface, for, rendering, html]","[rendering, html, with, the, htmltextwriter, i...","{interface, rendering, html, fluent, for}","{interface, rendering, html, fluent, for}","{interface, rendering, html, fluent, for}"
2,2,414714,2009-01-05 22:21:04,"[c++, compiler-construction, makefile, g++, mu...",159848,7,8,181,quick question: what is the compiler flag to a...,197,0,"[compiling, with, g++, using, multiple, cores]","[quick, question, :, what, is, the, compiler, ...","{multiple, g++, using, compiling, with, cores}","{multiple, g++, using, compiling, with, cores}","{multiple, g++, compiling, using, with, cores}"
3,3,415452,2009-01-06 04:26:16,"[c, oop, object]",82624,6,23,161,what would be a set of nifty preprocessor hack...,477,0,"[object-orientation, in, c]","[what, would, be, a, set, of, nifty, preproces...","{c, in, object-orientation}","{c, in, object-orientation}","{c, in, object-orientation}"
4,4,415515,2009-01-06 04:55:55,"[c++, csv]",147860,7,9,51,"pretty self-explanatory, i tried google and go...",181,0,"[how, can, i, read, and, manipulate, csv, file...","[pretty, self-explanatory, ,, i, tried, google...","{can, i, file, in, manipulate, data, ?, and, r...","{can, i, file, in, manipulate, data, ?, and, r...","{and, data, how, can, file, in, manipulate, ?,..."


In [31]:
df = df.drop_duplicates(subset = 'Id')

In [32]:
df.shape

(101611, 16)

In [33]:
df.isna().sum()

index                    0
Id                       0
CreationDate             0
Tags                     0
ViewCount                0
CommentCount             0
AnswerCount              0
Score                    0
Clean Body               0
body_length              0
code_snippets            0
clean tokenized title    0
clean tokenized body     0
title set                0
body set                 0
union body title set     0
dtype: int64

##### Export to CSV

In [34]:
cd '/content/drive/Shared drives/CIS522'

/content/drive/Shared drives/CIS522


In [35]:
df.to_csv('final_final_df.csv', index = False)

For some really weird reason, when the df is saved to a csv it duplicates a ton of stuff and goes crazy so you have to run the function below to be able to use the cleaned df from the one saved.

In [36]:
def fix_uploaded_csv(df):
  df = df.drop_duplicates(subset = 'Id')
  df = df.reset_index()
  df = df.drop(columns = ['index'])
  df = df.dropna(how = 'any')
  print(df.shape)
  return df

#### export to pkl

In [37]:
df.head(5)

,index,Id,CreationDate,Tags,ViewCount,CommentCount,AnswerCount,Score,Clean Body,body_length,code_snippets,clean tokenized title,clean tokenized body,title set,body set,union body title set
0,0,414336,2009-01-05 20:09:49,"[sql, sql-server, optimization]",3368,6,9,7,i know this has something to do with parameter...,1578,2,"[why, does, the, sqlserver, optimizer, get, so...","[i, know, this, has, something, to, do, with, ...","{sqlserver, so, does, why, confused, with, the...","{sqlserver, so, does, why, confused, with, the...","{so, why, confused, with, optimizer, get, para..."
1,1,414605,2009-01-05 21:33:21,"[.net, html, render, web-controls, htmltextwri...",4557,7,4,10,rendering html with the htmltextwriter is not ...,5117,4,"[fluent, interface, for, rendering, html]","[rendering, html, with, the, htmltextwriter, i...","{interface, rendering, html, fluent, for}","{interface, rendering, html, fluent, for}","{interface, rendering, html, fluent, for}"
2,2,414714,2009-01-05 22:21:04,"[c++, compiler-construction, makefile, g++, mu...",159848,7,8,181,quick question: what is the compiler flag to a...,197,0,"[compiling, with, g++, using, multiple, cores]","[quick, question, :, what, is, the, compiler, ...","{multiple, g++, using, compiling, with, cores}","{multiple, g++, using, compiling, with, cores}","{multiple, g++, compiling, using, with, cores}"
3,3,415452,2009-01-06 04:26:16,"[c, oop, object]",82624,6,23,161,what would be a set of nifty preprocessor hack...,477,0,"[object-orientation, in, c]","[what, would, be, a, set, of, nifty, preproces...","{c, in, object-orientation}","{c, in, object-orientation}","{c, in, object-orientation}"
4,4,415515,2009-01-06 04:55:55,"[c++, csv]",147860,7,9,51,"pretty self-explanatory, i tried google and go...",181,0,"[how, can, i, read, and, manipulate, csv, file...","[pretty, self-explanatory, ,, i, tried, google...","{can, i, file, in, manipulate, data, ?, and, r...","{can, i, file, in, manipulate, data, ?, and, r...","{and, data, how, can, file, in, manipulate, ?,..."


## EDA 

In [38]:
df['Clean Body'].apply(lambda text: len(text))

0         1578
1         5117
2          197
3          477
4          181
          ... 
101606     692
101607     600
101608    2838
101609    1953
101610     631
Name: Clean Body, Length: 101611, dtype: int64

In [39]:
df.head(10)

,index,Id,CreationDate,Tags,ViewCount,CommentCount,AnswerCount,Score,Clean Body,body_length,code_snippets,clean tokenized title,clean tokenized body,title set,body set,union body title set
0,0,414336,2009-01-05 20:09:49,"[sql, sql-server, optimization]",3368,6,9,7,i know this has something to do with parameter...,1578,2,"[why, does, the, sqlserver, optimizer, get, so...","[i, know, this, has, something, to, do, with, ...","{sqlserver, so, does, why, confused, with, the...","{sqlserver, so, does, why, confused, with, the...","{so, why, confused, with, optimizer, get, para..."
1,1,414605,2009-01-05 21:33:21,"[.net, html, render, web-controls, htmltextwri...",4557,7,4,10,rendering html with the htmltextwriter is not ...,5117,4,"[fluent, interface, for, rendering, html]","[rendering, html, with, the, htmltextwriter, i...","{interface, rendering, html, fluent, for}","{interface, rendering, html, fluent, for}","{interface, rendering, html, fluent, for}"
2,2,414714,2009-01-05 22:21:04,"[c++, compiler-construction, makefile, g++, mu...",159848,7,8,181,quick question: what is the compiler flag to a...,197,0,"[compiling, with, g++, using, multiple, cores]","[quick, question, :, what, is, the, compiler, ...","{multiple, g++, using, compiling, with, cores}","{multiple, g++, using, compiling, with, cores}","{multiple, g++, compiling, using, with, cores}"
3,3,415452,2009-01-06 04:26:16,"[c, oop, object]",82624,6,23,161,what would be a set of nifty preprocessor hack...,477,0,"[object-orientation, in, c]","[what, would, be, a, set, of, nifty, preproces...","{c, in, object-orientation}","{c, in, object-orientation}","{c, in, object-orientation}"
4,4,415515,2009-01-06 04:55:55,"[c++, csv]",147860,7,9,51,"pretty self-explanatory, i tried google and go...",181,0,"[how, can, i, read, and, manipulate, csv, file...","[pretty, self-explanatory, ,, i, tried, google...","{can, i, file, in, manipulate, data, ?, and, r...","{can, i, file, in, manipulate, data, ?, and, r...","{and, data, how, can, file, in, manipulate, ?,..."
5,5,415801,2009-01-06 08:18:28,"[php, memory-management, memory-limit]",629744,9,23,311,"this error message is being presented, any sug...",153,0,"[allowed, memory, size, of, 33554432, bytes, e...","[this, error, message, is, being, presented, ,...","{size, 33554432, 43148176, ), allowed, in, byt...","{size, 33554432, 43148176, ), allowed, in, byt...","{33554432, ), (, allocate, php, to, size, 4314..."
6,6,452859,2009-01-17 05:55:54,"[sql, sql-server, tsql, insert]",2836553,14,4,1788,"i have multiple set of data to insert at once,...",405,4,"[inserting, multiple, rows, in, a, single, sql...","[i, have, multiple, set, of, data, to, insert,...","{in, sql, multiple, inserting, a, ?, query, ro...","{in, sql, multiple, inserting, a, ?, query, ro...","{in, sql, multiple, inserting, a, ?, query, ro..."
7,7,417396,2009-01-06 17:18:20,"[python, syntax-rules]",9643,7,3,10,"originally asked (wrongly) what does ""|"" mean...",171,0,"[what, does, the, “, |, ”, sign, mean, in, pyt...","[originally, asked, (, wrongly, ), what, does,...","{what, in, sign, |, “, does, mean, the, ?, ”, ...","{what, in, sign, |, “, does, mean, the, ?, ”, ...","{what, “, mean, ”, python, in, sign, |, does, ..."
8,8,422600,2009-01-07 23:21:21,"[c++, operator-overloading]",2884,6,4,7,is there a canonical or recommended pattern fo...,990,2,"[canonical, operator, overloading, ?]","[is, there, a, canonical, or, recommended, pat...","{canonical, operator, ?, overloading}","{canonical, operator, ?, overloading}","{operator, ?, overloading, canonical}"
9,9,452368,2009-01-16 23:07:47,[c#],30778,8,5,97,"okay, this may be a dumb question, but i have ...",860,0,"[why, do, lowercase, and, uppercase, versions,...","[okay, ,, this, may, be, a, dumb, question, ,,...","{string, i, versions, should, uppercase, lower...","{string, i, versions, should, uppercase, lower...","{why, do, and, use, string, versions, should, ..."


DataLoaders

In [47]:
df[:75000].tail(5)

,index,Id,CreationDate,Tags,ViewCount,CommentCount,AnswerCount,Score,Clean Body,body_length,code_snippets,clean tokenized title,clean tokenized body,title set,body set,union body title set
74995,11609,32010501,2015-08-14 12:47:37,"[java, floating-point, comparison-operators]",1542,7,2,8,why is java inconsistent in comparing -0.0 and...,799,2,"[java, -, comparison, of, positive, and, negat...","[why, is, java, inconsistent, in, comparing, -...","{java, negative, zeros, comparison, positive, ...","{java, negative, zeros, comparison, positive, ...","{java, negative, zeros, comparison, positive, ..."
74996,11610,32011188,2015-08-14 13:23:53,"[java, performance, for-loop, benchmarking, by...",1062,7,2,28,"the ""performance tips"" section in the android ...",3037,13,"[does, using, an, intermediate, variable, inst...","[the, ``, performance, tips, '', section, in, ...","{intermediate, an, faster, using, instead, doe...","{intermediate, an, faster, using, instead, doe...","{intermediate, an, array.length, of, variable,..."
74997,11611,32011248,2015-08-14 13:26:34,"[javascript, jquery, jsp, highcharts]",55843,13,5,32,getting this error while running a highcharts ...,1090,3,"[highcharts, :, uncaught, typeerror, :, $, (, ...","[getting, this, error, while, running, a, high...","{highcharts, ), typeerror, $, uncaught, (, not...","{highcharts, ), typeerror, $, uncaught, (, not...","{highcharts, ), (, a, :, typeerror, $, uncaugh..."
74998,11612,33021997,2015-10-08 17:09:40,"[c#, .net, visual-studio, visual-studio-2015]",1767,6,2,7,xml comments (before methods etc.) used to hid...,316,2,"[hide, xml, comments, in, visual, studio, 2015]","[xml, comments, (, before, methods, etc, ., ),...","{studio, in, 2015, visual, hide, comments, xml}","{studio, in, 2015, visual, hide, comments, xml}","{studio, in, 2015, visual, hide, comments, xml}"
74999,11613,33022243,2015-10-08 17:22:36,"[ajax, image, php-gd]",787,11,5,8,i try to create a captcha code image with php ...,1494,3,"[send, string, to, php, gd, through, ajax, req...","[i, try, to, create, a, captcha, code, image, ...","{string, request, send, gd, php, to, through, ...","{string, request, send, gd, php, to, through, ...","{string, request, send, gd, php, to, through, ..."


In [44]:
training_data = df[:75000]
test_data = df[75001:]

In [41]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)